In [ ]:
import pandas as pd

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.width", None)

### Загрузка таблиц

In [ ]:
df_cashflow = pd.read_csv("../data/processed/cashflow.csv")
df_cashflow

In [ ]:
df_stocks = pd.read_csv("../data/processed/stocks.csv")
df_stocks

In [ ]:
df_catalog = pd.read_csv("../data/processed/catalog.csv")
df_catalog

In [ ]:
df_contracts = pd.read_csv("../data/processed/contracts.csv")
df_contracts

### Изучение возможности сопоставить имена активов в ведомостях с именами в справочнике КПГЗ

Проблема такова:
1. В ведомостях не используются никакие коды активов
2. Эти коды необходимы как для группирования товаров перед прогнозированием, так и, вероятнее всего, для отправки JSON-файлов
3.Коды предоставлены только в справочнике, но единственное поле, содержащееся везде - Название Актива

То есть нужно решить проблему сопоставления Названий из ведомостей с Названиями из справочника, но:
1. В справочнике может не содержаться записи о каком-то активе
2. Справочник может содержать дополнительную информацию, например, характеристики актива
3. Справочник и ведомости подвержены человеческому фактору: 
    - Лишние символы (пробелы, запятые, точки и т.п.) 
    - Лексические ошибки

**Задача: сопоставить максимально возможному числу записей из ведомостей их коды КПГЗ и СПГЗ из справочника.**

Ниже - поиск подхода

In [ ]:
# Подготовка серий
cashflow_names: pd.Series = pd.Series(df_cashflow.Name.unique())
stocks_names: pd.Series = pd.Series(df_stocks.Name.unique())
all_names: pd.Series = pd.Series(pd.concat([cashflow_names, stocks_names]).unique())
catalog_names: pd.Series = pd.Series(df_catalog.Name.unique())

In [ ]:
# Удаление небуквенных символов + полное совпадение
all_names_clean = all_names.str.replace("[^a-zA-Zа-яА-Я]", ' ', regex=True)
catalog_names_clean = catalog_names.str.replace("[^a-zA-Zа-яА-Я]", '', regex=True)

full_matches = all_names[all_names.isin(catalog_names)]
percent1 = len(full_matches) / len(all_names) * 100
print(f"Полных совпадений: {len(full_matches)}/{len(all_names)} ({percent1:.2f}%)")

In [ ]:
# Совпадения хотя бы по одному слову
def get_names_by_is_any_word_in(search: pd.Series, reference: pd.Series):
    result = []
    for s_name in search:
        s_words = set(s_name.split())
        for r_name in reference:
            r_words = set(r_name.split())
            if s_words & r_words:
               result.append(s_name)
               break
    return result

partial_matches = get_names_by_is_any_word_in(all_names, catalog_names)
percent2 = len(partial_matches) / len(all_names) * 100
print(f"Совпадений хотя бы по слову: {len(partial_matches)}/{len(all_names)} ({percent2:.2f}%)")

### Выводы
Названия часто содержат в себе единицы измерения и числа. Придумывать алгоритм очистки данных и последующего поиска с нуля - долго и нерационально, из-за такой проблемы частичные совпадения дают резкий рост совпадений. Хороший вариант - применение алгоритмов нечеткого поиска

### Изучение нечеткого сравнения. Расстояния Левенштейна и библиотека fuzzywuzzy
Нужные библиотеки и их лицензии (все с открытым лицензированием):
- [thefuzz](https://github.com/seatgeek/thefuzz]) - MIT - Нечеткое сравнение строк с применением расстояний Левенштейна
- [rapidfuzz](https://github.com/rapidfuzz/RapidFuzz) - MIT - Основа thefuzz
- [levenshtein](https://github.com/rapidfuzz/Levenshtein) - GNU GPl-2.0 - Реализация расчета расстояний на C, ускоряет расчеты в 4-10 раз

К базовым алгоритмам сравнения библиотеки относятся

| Функция                  | Описание                                                            |
|--------------------------|---------------------------------------------------------------------|
| ratio                    | Полностью сравнивает строки между собой                             |
| partial_ratio            | Частично сравнивает строки. Чувствителен к регистру                 |
| token_sort_ratio         | Сравнивает токены строк. Независим от порядка и регистра            |
| token_set_ratio          | Сравнивает уникальные токены строк. Независим от порядка и регистра |
| partial_token_sort_ratio | Частичный token_sort_ratio                                          |
| partial_token_set_ratio  | Частичный token_set                                                 |

Так же есть более сложные и интересные WRatio, UWRatio, QRatio, UQRatio.
В статьях часто рекомендуют использовать именно WRatio

In [ ]:
from fuzzywuzzy import fuzz

In [ ]:
# Пример работы с fuzzywuzzy

# ratio() Полное сравнение строк
print("ratio")
print("'Привет мир', 'Привет мир':", fuzz.ratio("Привет мир", "Привет мир"))
print("'Привет мир', 'Привет кир':", fuzz.ratio("Привет мир", "Привет кир"))
print()

# partial_ratio() Частичное сравнение строк. Вроде поиска подстроки с учетом регистра
print("partial_ratio")
print("'Привет мир', 'Привет мир!!!':", fuzz.partial_ratio("Привет мир", "Привет мир!!!"))
print("'Привет мир', 'Всем своим салам, остальным - Привет мир!!!':", fuzz.partial_ratio("Привет мир", "Всем своим салам, остальным - Привет мир!!!"))
print("'Привет мир', 'привет мир':", fuzz.partial_ratio("Привет мир", "привет мир"))
print()

# token_sort_ratio() Полное сравнение по токенам. Не зависит от порядка слов и регистра символов
print("token_sort_ratio")
print("'Привет наш мир', 'мир наш Привет':", fuzz.token_sort_ratio("Привет наш мир", "мир наш Привет"))
print("'Привет наш мир', 'мир наш любимый Привет':", fuzz.token_sort_ratio("Привет наш мир", "мир наш любимый Привет"))
print("'1 2 Привет наш мир', '1 мир наш 2 ПриВЕт':", fuzz.token_sort_ratio("1 2 Привет наш мир", "1 мир наш 2 ПриВЕт"))
print("'1 2 Привет наш мир', '1 мир наш 2 ПриВЕт Лишнее Слово!':", fuzz.token_sort_ratio("1 2 Привет наш мир", "1 мир наш 2 ПриВЕт Лишнее Слово!"))
print()

# token_set_ratio() Полное сравнение по токенам. Не зависит от порядка и регистра символов, не учитывает повторяющиеся слова
print("token_set_ratio")
print("'Привет наш мир', 'мир мир наш наш наш ПриВЕт':", fuzz.token_set_ratio("Привет наш мир", "мир, мир, наш наш наш ПриВЕт"))

# WRatio()
print("WRatio")
print("'Привет наш мир', '!ПриВЕт наш мир!':", fuzz.WRatio("Привет наш мир", "!ПриВЕт наш мир!"))

In [ ]:
# Выборки для тестирования
data1: pd.Series = cashflow_names[:100]
key1 = "Конверт C4"
target1 = 1

data2: pd.Series = stocks_names[:100]
key2 = "Клей Момент"
target2 = 2

data3: pd.Series = catalog_names[:100]
key3 = "Доска brauberg"
target3 = 4 

datasets = [
    [1, key1, data1, target1],
    [2, key2, data2, target2],
    [3, key3, data3, target3]
]


In [ ]:
# Методы сравнения. partial_ratio() и ratio() явно не интересны
methods = [
    fuzz.UWRatio, fuzz.WRatio,
    fuzz.QRatio, fuzz.UQRatio,
    fuzz.token_sort_ratio, fuzz.token_set_ratio,
    fuzz.partial_token_sort_ratio, fuzz.partial_token_set_ratio
]

In [ ]:
# Перебор методов и порога сравнения для всех наборов
error_tolerance = 1
print("Допустимая погрешность:", error_tolerance)
for method in methods:
    for param in range(100):
        result = []
        # Проходим по датасетам, если всё в пределах погрешности - сохраняем результат
        for i, key, data, target in datasets:
            matches = data[data.apply(lambda x: method(x, key)) > param]
            if (target - error_tolerance) <= len(matches) <= (target + error_tolerance):
                result.append([method, param, matches, key, data, target])
            else:
                result = []
                break
        else:
            print(f"{result[0][0].__name__} > {result[0][1]}:")
            for x in result:
                print(f"\t{len(x[2])}/{x[-1]} of {x[3]}")